# Preliminaries

In [ ]:
suppressPackageStartupMessages(library(tidyverse))
library(patchwork)

if (!require(foreach)) install.packages("foreach")
library(foreach)
if (!require(iterators)) install.packages("iterators")
library(iterators)
if (!require(mediation)) install.packages("mediation")
library(mediation)

In [ ]:
ws_namespace <- Sys.getenv("WORKSPACE_NAMESPACE")
ws_name <- Sys.getenv("WORKSPACE_NAME")
ws_bucket <- Sys.getenv("WORKSPACE_BUCKET")

In [ ]:
theme_set(theme_bw())

## Read in phenotypic, genotypic, and metabolomic data

* Phenotype data come from integrating inputs from dbGaP (/PIC-SURE), MESA investigators, and metadata from metabolomic preprocessing.
* Genotype data come from dbGaP (Freeze 9b TOPMed-wide genotype VCF files).
* Metabolomic data come directly from MESA (originally) followed by an extensive QC and preprocessing effort led by Magdalena Sevilla-Gonzalez and Paul Hanson.

In [ ]:
analysis_df_lcms <- read_csv("analysis/analysis_df_lcms.csv", col_types=cols())
names(analysis_df_lcms)

In [ ]:
snp_info_df <- read_csv("genotypes/snp_info.csv", col_types=cols())
exposures <- c("pa", "pa_bin", "mod_pa", "vig_pa")

basic_covars <- c("site", "gender_f0m1", "age")
covar_sets <- list(
    basic = basic_covars,
    add_ses = c(basic_covars, "ses_score", "income_cat"),
    add_ses_HL = c(basic_covars, "ses_score", "income_cat", 
                              "drinks_per_week", "smoking", "ahei_score", "dash_score"),
    add_ses_HL_gPC = c(basic_covars, "ses_score", "income_cat", 
                              "drinks_per_week", "smoking", "ahei_score", "dash_score",
                       paste0("gPC", 1:5)),
    add_ses_HL_race = c(basic_covars, "ses_score", "income_cat", 
                              "drinks_per_week", "smoking", "ahei_score", "dash_score",
                   "race")
)

In [ ]:
metabs <- readRDS("metabolites/analysis_metabolites.rds")

# Run metabolome-wide association studies

A few details on the analysis models:
* Based on prior investigations (see analysis-prep notebook), it appears that adjustment for age, sex, study site, and genetic principal components (for now, the top five) keep genomic inflation under control.
* Tahir et al. 2022 (Nat. Comm.) conducted a comprehensive set of mQTL tests in MESA and other studies. They adjusted for age, sex, 10 genetic PCs, and the genomic relationship matrix using a linear mixed model (with metabolites pre-adjusted for the same covariates plus batch then inverse-normal transformed).

In [ ]:
metab_bonferroni <- 0.05 / ncol(metabs)

In [ ]:
qtl_covars <- covar_sets$add_ses_HL_gPC
e_proxy_covars <- covar_sets$add_ses_HL_gPC

## QTL tests

For each SNP of interest, we will scan the metabolome to find associated metabolites (for downstream mediation testing).

In [ ]:
test_qtl <- function(g, m, covars) {
    form_str <- paste0("m ~ g + ", paste(covars, collapse=" + "))
    lm(as.formula(form_str), data=analysis_df_lcms) %>%
        broom::tidy() %>%
        filter(term == "g") %>%
        select(-term)
}

run_qtl_mwas <- function(rsID, covars) {
    foreach(
        m=iterators::iter(metabs, by="col")
    ) %do% {
        tryCatch(
            test_qtl(analysis_df_lcms[[rsID]], m, covars),
            error=function(e) tibble(estimate=NA, p.value=NA)
        )
    } %>%
        setNames(colnames(metabs)) %>%
        bind_rows(.id="metabolite")
}

In [ ]:
# qtl_mwas_res_df <- lapply(snp_info_df$rsID, run_qtl_mwas, qtl_covars) %>%
#     setNames(snp_info_df$rsID) %>%
#     bind_rows(.id="rsID")

# write_csv(qtl_mwas_res_df, "analysis/qtl_mwas_res.csv")

In [ ]:
qtl_mwas_res_df <- read_csv("analysis/qtl_mwas_res.csv", col_types=cols())

### Any different results in AFR samples?

In [ ]:
# analysis_df_lcms_afr <- filter(analysis_df_lcms, race == "r4")
# metabs_afr <- metabs[analysis_df_lcms$race == "r4", ]

# test_qtl_afr <- function(g, m, covars) {
#     form_str <- paste0("m ~ g + ", paste(covars, collapse=" + "))
#     lm(as.formula(form_str), data=analysis_df_lcms_afr) %>%
#         broom::tidy() %>%
#         filter(term == "g") %>%
#         select(-term)
# }
# run_qtl_mwas_afr <- function(rsID, covars) {
#     foreach(
#         m=iterators::iter(metabs_afr, by="col")
#     ) %do% {
#         tryCatch(
#             test_qtl_afr(analysis_df_lcms_afr[[rsID]], m, covars),
#             error=function(e) tibble(estimate=NA, p.value=NA)
#         )
#     } %>%
#         setNames(colnames(metabs)) %>%
#         bind_rows(.id="metabolite")
# }

In [ ]:
# qtl_mwas_res_df_afr <- lapply("rs77810251", run_qtl_mwas_afr, qtl_covars) %>%
#     setNames("rs77810251") %>%
#     bind_rows(.id="rsID")

# write_csv(qtl_mwas_res_df_afr, "analysis/qtl_mwas_res_afr.csv")

In [ ]:
# qtl_mwas_res_df_afr <- read_csv("analysis/qtl_mwas_res_afr.csv", col_types=cols())

In [ ]:
# qtl_mwas_res_df_afr %>%
#     mutate(q = p.adjust(p.value, method="BH")) %>%
#     arrange(p.value) %>%
#     head()

## E proxy tests

For each exposure of interest, we will scan the metabolome to find associated metabolites (for downstream mediation testing).

In [ ]:
test_e_proxy <- function(e, m, covars) {
    form_str <- paste0("m ~ e + ", paste(covars, collapse=" + "))
    lm(as.formula(form_str), data=analysis_df_lcms) %>%
        broom::tidy() %>%
        filter(term == "e") %>%
        select(-term)
}

run_e_proxy_mwas <- function(e, covars) {
    foreach(
        m=iterators::iter(metabs, by="col")
    ) %do% {
        tryCatch(
            test_e_proxy(analysis_df_lcms[[e]], m, covars),
            error=function(e) tibble(estimate=NA, p.value=NA)
        )
    } %>%
        setNames(colnames(metabs)) %>%
        bind_rows(.id="metabolite")
}

In [ ]:
# e_proxy_mwas_res_df <- lapply(exposures, run_e_proxy_mwas, e_proxy_covars) %>%
#     setNames(exposures) %>%
#     bind_rows(.id="exposure")

# write_csv(e_proxy_mwas_res_df, "analysis/e_proxy_mwas_res.csv")

In [ ]:
e_proxy_mwas_res_df <- read_csv("analysis/e_proxy_mwas_res.csv", col_types=cols()) %>%
    filter(exposure %in% c("pa", "vig_pa"))

In [ ]:
e_proxy_mwas_res_df %>%
    arrange(p.value)

## Annotate summary statistics

In [ ]:
met_info_df <- read_csv("PH_files/met_info_v12.csv", col_types=cols()) %>%
    dplyr::select(metabolite=Compound_Id_MESA, metabolite_name=Name)

qtl_mwas_res_df <- left_join(qtl_mwas_res_df, met_info_df, by="metabolite")
e_proxy_mwas_res_df <- left_join(e_proxy_mwas_res_df, met_info_df, by="metabolite")

# Visualize results

In [ ]:
calc_lambda <- function(x, p=0.5){
  # Calculate genomic inflation lambda value
  x <- x[!is.na(x)]
  x.quantile <- quantile(x, p)
  round(qchisq(1 - x.quantile, 1) / qchisq(p, 1), 2)
}

qtl_lambda <- calc_lambda(qtl_mwas_res_df$p.value)
qtl_qq_plt <- qtl_mwas_res_df %>%
    arrange(desc(p.value)) %>%
    mutate(nlp = -log10(p.value),
           exp_nlp = rev(-log10(ppoints(nrow(.))))) %>%
    ggplot(aes(x=exp_nlp, y=nlp, color=rsID)) +
    geom_point() +
    geom_abline(slope=1, intercept=0, 
               linetype="dashed", color="black") +
    annotate("text", x=-Inf, y=Inf, 
             hjust=-0.5, vjust=2, 
             label=paste("lambda", "==", qtl_lambda),
             parse=TRUE) +
    labs(x=expression(-log[10] * "(p) - expected"), y=expression(-log[10] * "(p) - observed"),
         title="QTL MWAS Q-Q plot")

e_proxy_lambda <- calc_lambda(e_proxy_mwas_res_df$p.value)
e_proxy_qq_plt <- e_proxy_mwas_res_df %>%
    arrange(desc(p.value)) %>%
    mutate(nlp = -log10(p.value),
           exp_nlp = rev(-log10(ppoints(nrow(.))))) %>%
    ggplot(aes(x=exp_nlp, y=nlp, color=exposure)) +
    geom_point() +
    geom_abline(slope=1, intercept=0, 
               linetype="dashed", color="black") +
    annotate("text", x=-Inf, y=Inf, 
             hjust=-0.5, vjust=2, 
             label=paste("lambda", "==", e_proxy_lambda),
             parse=TRUE) +
    labs(x=expression(-log[10] * "(p) - expected"), y=expression(-log[10] * "(p) - observed"),
         title="E-proxy MWAS Q-Q plot")

options(repr.plot.width=12, repr.plot.height=5)

qtl_qq_plt | e_proxy_qq_plt

In [ ]:
qtl_volcano_plt <- qtl_mwas_res_df %>%
    mutate(nlp = -log10(p.value)) %>%
    ggplot(aes(x=estimate, y=nlp)) +
    geom_point() +
    geom_hline(aes(yintercept=-log10(metab_bonferroni)), 
               linetype="dashed", color="gray") +
    labs(x="Regression estimate", y="-log10(P)") +
    facet_wrap(vars(rsID), nrow=1, scales="free")

e_proxy_volcano_plt <- e_proxy_mwas_res_df %>%
    mutate(nlp = -log10(p.value)) %>%
    ggplot(aes(x=estimate, y=nlp)) +
    geom_point() +
    geom_hline(aes(yintercept=-log10(metab_bonferroni)), 
               linetype="dashed", color="gray") +
    labs(x="Regression estimate", y="-log10(P)") +
    facet_wrap(vars(exposure), scales="free")

options(repr.plot.width=16, repr.plot.height=5)

qtl_volcano_plt
e_proxy_volcano_plt

# Follow up on top metabolites

## Collect significant metabolites

We use a false discovery rate (FDR) correction threshold of q < 0.05 using the Benjamini-Hochberg method.

In [ ]:
metab_pca_fit <- prcomp(metabs)
metab_eigenvals <- metab_pca_fit$sdev^2
n_eff_metabs <- sum(metab_eigenvals) ** 2 / sum(metab_eigenvals ** 2)
round(n_eff_metabs, 1)

In [ ]:
top_qtl_mwas_res_df <- qtl_mwas_res_df %>%
    mutate(q = p.adjust(p.value, method="BH"),
           sig_eff_bonferroni = p.value < (0.05 / n_eff_metabs)) %>%
    arrange(p.value) %>%
    filter(sig_eff_bonferroni)

top_e_proxy_mwas_res_df <- e_proxy_mwas_res_df %>%
    mutate(q = p.adjust(p.value, method="BH"),
           sig_eff_bonferroni = p.value < (0.05 / n_eff_metabs)) %>%
    arrange(p.value) %>%
    filter(sig_eff_bonferroni)

top_mwas_res_df <- bind_rows(list(
    qtl = top_qtl_mwas_res_df,
    e_proxy = top_e_proxy_mwas_res_df
), .id="type")

In [ ]:
top_mwas_res_df

## Test main effect mediation

In [ ]:
test_mediation <- function(x, m, y, covars, df, n_sims=100) {
    med_form_str <- paste0("m ~ x + ", paste(covars, collapse=" + "))
    med_fit <- lm(as.formula(med_form_str), data=df)
    out_form_str <- paste0("y ~ m + x + ", paste(covars, collapse=" + "))
    out_fit <- lm(as.formula(out_form_str), data=df)
    med_out <- mediation::mediate(med_fit, out_fit, 
                       treat="x", mediator="m",
                       robustSE=TRUE, sims=n_sims)
    summary(med_out)
}

In [ ]:
mediation_analysis_df_lcms <- analysis_df_lcms %>%
    filter(!is.na(pa),
           !is.na(vig_pa),
           !is.na(hdl_log))

In [ ]:
pa_mediation_res_df <- tibble(
    metab = top_e_proxy_mwas_res_df$metabolite[top_e_proxy_mwas_res_df$exposure == "pa"]
) %>%
  rowwise() %>%
  mutate(med_fit = list(
      test_mediation(mediation_analysis_df_lcms$pa, 
               metabs[match(mediation_analysis_df_lcms$mesa_id, analysis_df_lcms$mesa_id),
                      metab], 
               mediation_analysis_df_lcms$hdl_log, 
               covar_sets$basic,
               mediation_analysis_df_lcms,
               n_sims=100)
  )) %>%
    ungroup() %>%
    mutate(med_prop = map_dbl(med_fit, function(mf) mf$n0),
           acme_p = map_dbl(med_fit, function(mf) mf$d0.p)) %>%
    dplyr::select(-med_fit)

In [ ]:
vig_pa_mediation_res_df <- tibble(
    metab = top_e_proxy_mwas_res_df$metabolite[top_e_proxy_mwas_res_df$exposure == "vig_pa"]
) %>%
  rowwise() %>%
  mutate(med_fit = list(
      test_mediation(mediation_analysis_df_lcms$vig_pa, 
               metabs[match(mediation_analysis_df_lcms$mesa_id, analysis_df_lcms$mesa_id),
                      metab], 
               mediation_analysis_df_lcms$hdl_log, 
               covar_sets$basic,
               mediation_analysis_df_lcms,
               n_sims=100)
  )) %>%
    ungroup() %>%
    mutate(med_prop = map_dbl(med_fit, function(mf) mf$n0),
           acme_p = map_dbl(med_fit, function(mf) mf$d0.p)) %>%
    dplyr::select(-med_fit)

In [ ]:
pa_mediation_res_df
vig_pa_mediation_res_df

## Export

In [ ]:
write(n_eff_metabs, "analysis/n_eff_metabolites.txt")
write(top_qtl_mwas_res_df$metabolite, 
      "analysis/top_qtl_metabolites.txt")
write(top_e_proxy_mwas_res_df$metabolite[top_e_proxy_mwas_res_df$exposure == "pa"],
      "analysis/top_pa_metabolites.txt")
write(top_e_proxy_mwas_res_df$metabolite[top_e_proxy_mwas_res_df$exposure == "vig_pa"],
      "analysis/top_vig_pa_metabolites.txt")
write_csv(top_mwas_res_df, "analysis/top_mwas_res.csv")

system(paste0("gsutil cp -r analysis ", ws_bucket, "/"))

# Plots for presentation

In [ ]:
# qtl_lambda <- calc_lambda(qtl_mwas_res_df$p.value)
# qtl_qq_plt <- qtl_mwas_res_df %>%
#     arrange(desc(p.value)) %>%
#     mutate(nlp = -log10(p.value),
#            exp_nlp = rev(-log10(ppoints(nrow(.))))) %>%
#     ggplot(aes(x=exp_nlp, y=nlp, color=rsID)) +
#     geom_point() +
#     geom_abline(slope=1, intercept=0, 
#                linetype="dashed", color="black") +
#     annotate("text", x=-Inf, y=Inf, 
#              hjust=-0.5, vjust=2, 
#              label=paste("lambda", "==", qtl_lambda),
#              parse=TRUE) +
#     labs(x=expression(-log[10] * "(p) - expected"), y=expression(-log[10] * "(p) - observed")) +
#     theme(legend.title=element_blank(), legend.position="bottom") +
#     guides(color = guide_legend(nrow=2))

# e_proxy_lambda <- calc_lambda(e_proxy_mwas_res_df$p.value)
# e_proxy_qq_plt <- e_proxy_mwas_res_df %>%
#     arrange(desc(p.value)) %>%
#     mutate(nlp = -log10(p.value),
#            exp_nlp = rev(-log10(ppoints(nrow(.))))) %>%
#     ggplot(aes(x=exp_nlp, y=nlp, color=exposure)) +
#     geom_point() +
#     geom_abline(slope=1, intercept=0, 
#                linetype="dashed", color="black") +
#     scale_color_discrete(breaks=c("mod_vig_pa_bin", "smoking_current", "smoking_ever"),
#                          labels=c("Physical activity", "Smoking (current)", "Smoking (ever)"),
#                          name="Exposure") +
#     annotate("text", x=-Inf, y=Inf, 
#              hjust=-0.5, vjust=2, 
#              label=paste("lambda", "==", e_proxy_lambda),
#              parse=TRUE) +
#     labs(x=expression(-log[10] * "(p) - expected"), y=expression(-log[10] * "(p) - observed")) +
#     theme(legend.title=element_blank(), legend.position="bottom") +
#     guides(color = guide_legend(nrow=2))

# options(repr.plot.width=5, repr.plot.height=5)

# qtl_qq_plt
# e_proxy_qq_plt

In [ ]:
# e_proxy_mwas_res_df %>%
#     filter(exposure == "pa") %>%
#     arrange(desc(p.value)) %>%
#     mutate(nlp = -log10(p.value),
#            exp_nlp = rev(-log10(ppoints(nrow(.))))) %>%
#     ggplot(aes(x=exp_nlp, y=nlp, color=exposure)) +
#     geom_point() +
#     geom_abline(slope=1, intercept=0, 
#                linetype="dashed", color="black") +
#     scale_color_discrete(breaks=c("mod_vig_pa_bin", "smoking_current", "smoking_ever"),
#                          labels=c("Physical activity", "Smoking (current)", "Smoking (ever)"),
#                          name="Exposure") +
#     annotate("text", x=-Inf, y=Inf, 
#              hjust=-0.5, vjust=2, 
#              label=paste("lambda", "==", e_proxy_lambda),
#              parse=TRUE) +
#     labs(x=expression(-log[10] * "(p) - expected"), y=expression(-log[10] * "(p) - observed")) +
#     theme(legend.title=element_blank(), legend.position="bottom") +
#     guides(color = guide_legend(nrow=2))